# Battle of Neighborhoods in the city of Toronto


## Part 1 - Collecting Neighborhoods data

### Importing the necessary libraries


In [1]:
from bs4 import BeautifulSoup as bs 
import requests

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

import folium

from sklearn.cluster import KMeans

### Scrap data from the Wikipedia page

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = bs(page.content,"html.parser")

### Data Extraction from scraped data and creating a data frame.
##### 1.The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
##### 2.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

### Create a Dataframe 

In [4]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
print("Number of rows:",df.shape[0])
print("Number of Columns:",df.shape[1])

Number of rows: 103
Number of Columns: 3


## Part 2 - Finding the latitude and the longitude coordinates of each neighborhood. 

### Read the Geospatial_Coordinates dataset

In [6]:
Geospatial_data=pd.read_csv("E:/Projects/Coursera Capstone/data/Geospatial_Coordinates.csv")
Geospatial_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [7]:
Geospatial_data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
Geospatial_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the neighborhood data and the Geospatial_Coordinates data on the column PostalCode

In [8]:
df1 = df.merge(Geospatial_data, on="PostalCode", how="left")
df1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Part 3 - Clustering of Neighborhoods based on  the venue categories.

### 1. Find the co-ordinates of Toronto city.

In [9]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


### 2. Plot the map of Toronto city with the neighborhoods

In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

#### Observation
There are many neighborhoods in the Downtown Toronto Borough. Let's explore this borough and find the top venues where the people like to spend their time the most.

### 3. Plot a map of neighborhoods of Downtown Toronto.

In [11]:
Downtown_Toronto_Data = df1[df1['Borough']== 'Downtown Toronto'].reset_index(drop=True)

In [12]:
print(Downtown_Toronto_Data.shape)
Downtown_Toronto_Data.head()

(17, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(
    Downtown_Toronto_Data['Latitude'], 
    Downtown_Toronto_Data['Longitude'], 
    Downtown_Toronto_Data['Borough'], 
    Downtown_Toronto_Data['Neighborhood']):
    
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### 4. Collect all the venues in the surrounding area of neighborhoods using FourSquare api.

In [14]:
CLIENT_ID = 'O0UBOBVU3CF41S5KXLPHFCE4XDURYNTJJYLLOJP0XLEVKQAU' # your Foursquare ID
CLIENT_SECRET = 'EU00XNHQ5FTUB5EFUPJOABCRRXKYVTVHI14VASK2YGFFOTO1' # your Foursquare Secret
ACCESS_TOKEN = 'KVQASILUCQ3KJX0FQZO41ROG1TCP2VJL0NIFOPHNNBUEGG5W' # your FourSquare Access Token
VERSION = '20210720'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O0UBOBVU3CF41S5KXLPHFCE4XDURYNTJJYLLOJP0XLEVKQAU
CLIENT_SECRET:EU00XNHQ5FTUB5EFUPJOABCRRXKYVTVHI14VASK2YGFFOTO1


In [15]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(
    Downtown_Toronto_Data['Latitude'], 
    Downtown_Toronto_Data['Longitude'],
    Downtown_Toronto_Data['PostalCode'],
    Downtown_Toronto_Data['Borough'],
    Downtown_Toronto_Data['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [16]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1115, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


### 5. Finding the count of venues in each neighborhood 

In [17]:
venues_df.groupby(["PostalCode"]).count().reset_index()[['PostalCode','VenueCategory']]

,PostalCode,VenueCategory
0,M4W,4
1,M4X,45
2,M4Y,79
3,M5A,43
4,M5B,100
5,M5C,87
6,M5E,58
7,M5G,67
8,M5H,100
9,M5J,100


### Unique Venues in Downtown Toronto

In [18]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
venues_df.VenueCategory.unique()[:20]


There are 205 uniques categories.


array(['Bakery', 'Coffee Shop', 'Distribution Center', 'Restaurant',
       'Spa', 'Park', 'Pub', 'Gym / Fitness Center', 'Historic Site',
       'Chocolate Shop', 'Farmers Market', 'Performing Arts Venue',
       'Breakfast Spot', 'Dessert Shop', 'French Restaurant',
       'Mexican Restaurant', 'Theater', 'Yoga Studio', 'Shoe Store',
       'Event Space'], dtype=object)

### 6. Preparing data for the clustering

#### One hot encoding

In [19]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1115, 208)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group by postal codes to find the top 10 venues

In [20]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()
toronto_grouped

,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4W,Downtown Toronto,Rosedale,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.25,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
2,M4Y,Downtown Toronto,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,...,0.012658,0.012658,0.00,0.00,0.00,0.000000,0.00000,0.012658,0.000000,0.025316
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.046512,0.000000,0.00,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.023256
4,M5B,Downtown Toronto,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.01,0.00,0.00,0.000000,0.01000,0.010000,0.000000,0.000000
5,M5C,Downtown Toronto,St. James Town,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022989,...,0.011494,0.000000,0.00,0.00,0.00,0.011494,0.00000,0.000000,0.011494,0.000000
6,M5E,Downtown Toronto,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.017241,0.00000,0.000000,0.000000,0.000000
7,M5G,Downtown Toronto,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.014925,0.00000,0.000000,0.000000,0.014925
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.010000,0.000000,0.00,0.00,0.00,0.030000,0.00000,0.000000,0.000000,0.000000
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.00000,0.000000,0.010000,0.000000


#### Create new columns according to the number of top venues

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

#### Create a new dataframe for all the top 10 venues

In [22]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(17, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,Park,Playground,Trail,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",Coffee Shop,Bakery,Restaurant,Italian Restaurant,Pub,Pizza Place,Café,Sandwich Place,Market,Japanese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Ice Cream Shop,Hotel
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Café,Theater,Yoga Studio,Mexican Restaurant,Breakfast Spot,Restaurant
4,M5B,Downtown Toronto,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Lingerie Store,Middle Eastern Restaurant,Electronics Store,Ramen Restaurant
5,M5C,Downtown Toronto,St. James Town,Coffee Shop,Café,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,Bakery,Italian Restaurant,Seafood Restaurant,Beer Bar
6,M5E,Downtown Toronto,Berczy Park,Bakery,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Restaurant,Pharmacy,Cheese Shop,Seafood Restaurant,Pub
7,M5G,Downtown Toronto,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Spa,Salad Place
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",Coffee Shop,Clothing Store,Café,Hotel,Gym,Bar,Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Bookstore
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Brewery,Scenic Lookout,Park,Pizza Place


### Part 7- Implementing the KNN algorithm to create clusters of neighborhoods on similar venues

In [23]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 4, 0, 3, 2, 5])

In [24]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Downtown_Toronto_merged = Downtown_Toronto_Data.copy()

# add clustering labels
Downtown_Toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Downtown_Toronto_merged = Downtown_Toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(Downtown_Toronto_merged.shape)
Downtown_Toronto_merged # check the last columns!

(17, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Café,Theater,Yoga Studio,Mexican Restaurant,Breakfast Spot,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Lingerie Store,Middle Eastern Restaurant,Electronics Store,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,Bakery,Italian Restaurant,Seafood Restaurant,Beer Bar
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Bakery,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Restaurant,Pharmacy,Cheese Shop,Seafood Restaurant,Pub
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Spa,Salad Place
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Bank,Restaurant,Baby Store,Nightclub,Coffee Shop
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Clothing Store,Café,Hotel,Gym,Bar,Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Bookstore
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,2,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Brewery,Scenic Lookout,Park,Pizza Place
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,2,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,Bakery,Japanese Restaurant,Sushi Restaurant,Bar
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,2,Coffee Shop,Restaurant,Hotel,Café,Gym,Japanese Restaurant,Seafood Restaurant,Bakery,Deli / Bodega,American Restaurant


In [141]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
col = ['orange', 'green', 'red','yellow','blue','purple']


# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(
    Downtown_Toronto_merged['Latitude'],
    Downtown_Toronto_merged['Longitude'],
    Downtown_Toronto_merged['PostalCode'],
    Downtown_Toronto_merged['Borough'],
    Downtown_Toronto_merged['Neighborhood'], 
    Downtown_Toronto_merged['Cluster Labels']):
    
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=col[cluster],
        fill=True,
        fill_color=col[cluster],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

### Results

### Cluster 0

In [25]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 0, Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Lingerie Store,Middle Eastern Restaurant,Electronics Store,Ramen Restaurant
5,Christie,0,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Bank,Restaurant,Baby Store,Nightclub,Coffee Shop
6,"Richmond, Adelaide, King",0,Coffee Shop,Clothing Store,Café,Hotel,Gym,Bar,Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Bookstore
13,Rosedale,0,Park,Playground,Trail,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 1

In [26]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 1, Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Bakery,Pub,Park,Café,Theater,Yoga Studio,Mexican Restaurant,Breakfast Spot,Restaurant


### Cluster 2

In [27]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 2, Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,St. James Town,2,Coffee Shop,Café,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,Bakery,Italian Restaurant,Seafood Restaurant,Beer Bar
3,Berczy Park,2,Bakery,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Restaurant,Pharmacy,Cheese Shop,Seafood Restaurant,Pub
4,Central Bay Street,2,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Spa,Salad Place
7,"Harbourfront East, Union Station, Toronto Islands",2,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Brewery,Scenic Lookout,Park,Pizza Place
8,"Toronto Dominion Centre, Design Exchange",2,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,Bakery,Japanese Restaurant,Sushi Restaurant,Bar
9,"Commerce Court, Victoria Hotel",2,Coffee Shop,Restaurant,Hotel,Café,Gym,Japanese Restaurant,Seafood Restaurant,Bakery,Deli / Bodega,American Restaurant
10,"University of Toronto, Harbord",2,Café,Bookstore,Bar,Japanese Restaurant,Bakery,Yoga Studio,Italian Restaurant,Beer Bar,Beer Store,Sandwich Place
11,"Kensington Market, Chinatown, Grange Park",2,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Bakery,Park,Gaming Cafe,Burger Joint
15,"First Canadian Place, Underground city",2,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Asian Restaurant,Steakhouse,Seafood Restaurant,Deli / Bodega


### Cluster 3

In [28]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 3, Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"St. James Town, Cabbagetown",3,Coffee Shop,Bakery,Restaurant,Italian Restaurant,Pub,Pizza Place,Café,Sandwich Place,Market,Japanese Restaurant


### Cluster 4

In [29]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 4, Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"CN Tower, King and Spadina, Railway Lands, Har...",4,Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry


### Cluster 5

In [30]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 5, Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Church and Wellesley,5,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Ice Cream Shop,Hotel


### Conclusion
#### Downtown Toronto is a buzzing area filled with shopping area, restaurants, nightlife, parks, cultural venues and an eclectic mix of neighbourhoods. 
There are lots of coffee shops across the neighborhoods of Downtown Toronto. Let's explore the other places where people visit the most are 
- Cluster 0 - Park, Playground, 
- Cluster 1 - Bakery, Pub, Breakfast Spot, Theater
- Cluster 2 - Bar, Japanese Restaurant, Aquarium 
- Cluster 3 - Sushi Restaurant, Gay Bar, Yoga Studio	
- Cluster 4 - Airport Service, Harbor / Marina, Sculpture Garden, Boat or Ferry	
- Cluster 5 - Pizza Place, Italian Restaurant	